In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam, SGD, AdamW
import torchvision
from torchvision import models

import numpy as np
import time
import random
from tqdm.auto import tqdm
from PIL import Image
from os.path import join
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pandas as pd
import os
from torch.utils.data import DataLoader, sampler,Dataset
from sklearn.metrics import log_loss

In [2]:
class CFG:
    # ============== comp exp name =============
    comp_name = 'dog_breed_identification'

    # comp_dir_path = './'
    comp_dir_path = '/kaggle/input/'
    comp_folder_name = 'dog_breed_identification'
    # comp_dataset_path = f'{comp_dir_path}datasets/{comp_folder_name}/'
    comp_dataset_path = f'{comp_dir_path}{comp_folder_name}/'
    exp_name = 'exp_01'

    # ============== model cfg =============
    model_name = 'resnet50'
    in_chans = 16 
    # ============== training cfg =============
    trian_size = 10000
    
    size = 224
    batch_size = 32
    epochs = 50
    lr = 0.001

    # ============== set dataset path =============
    print('set dataset path')
    
    input_path = f'/kaggle/input/dog-breed-identification'
    
    outputs_path = f'/kaggle/working/outputs/{comp_name}/{exp_name}/'

    submission_dir = outputs_path + 'submissions/'
    submission_path = submission_dir + f'submission_{exp_name}.csv'

    model_dir = outputs_path + \
        f'{comp_name}-models/'

    log_dir = outputs_path + 'logs/'
    log_path = log_dir + f'{exp_name}.txt'

    num_workers = 4
    seed = 2

set dataset path


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
for dir in [CFG.model_dir, CFG.submission_dir, CFG.log_dir]:
    os.makedirs(dir, exist_ok=True)

In [5]:
def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 2

    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic

def init_logger(log_file):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger
set_seed()
Logger = init_logger(CFG.log_path)

In [6]:
label_path = join(CFG.input_path,'labels.csv')
label_csv = pd.read_csv(label_path)


from sklearn import preprocessing
label = preprocessing.LabelEncoder()
label_csv['breed'] = label.fit_transform(label_csv['breed']) 
print(label_csv)


                                     id  breed
0      000bec180eb18c7604dcecc8fe0dba07     19
1      001513dfcb2ffafc82cccf4d8bbaba97     37
2      001cdf01b096e06d78e9e5112d419397     85
3      00214f311d5d2247d5dfe4fe24b2303d     15
4      0021f9ceb3235effd7fcde7f7538ed62     49
...                                 ...    ...
10217  ffd25009d635cfd16e793503ac5edef0     18
10218  ffd3f636f7f379c51ba3648a9ff8254f     35
10219  ffe2ca6c940cddfee68fa3cc6c63213f      3
10220  ffe5f6d8e2bff356e9482a80a6e29aac     75
10221  fff43b07992508bc822f33d8ffd902ae     28

[10222 rows x 2 columns]


In [7]:
model = models.resnet50(pretrained=True)
model.to(device)

optimizer = Adam(model.parameters(), lr=CFG.lr)
scheduler = ReduceLROnPlateau(optimizer,'min',verbose=True,patience=4,factor=0.1)

criterion = nn.CrossEntropyLoss()


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [8]:
class DogsDataset(Dataset):
    def __init__(self, labels, root, train=True, transform=None):
        self.labels = labels
        self.root = CFG.input_path+root
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img_name = '{}.jpg'.format(self.labels.iloc[idx, 0])
        fullname = join(self.root, img_name)
        image = Image.open(fullname)
        labels = self.labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return [image, labels]

In [9]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])
mean, std = [0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]
composed_train = transforms.Compose([transforms.Resize((CFG.size,CFG.size)),
                                     transforms.RandomRotation(20), # Randomly rotate some images by 20 degrees
                                     transforms.RandomHorizontalFlip(0.1), # Randomly horizontal flip the images
                                     transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1, 
                                                            saturation = 0.1), 
                                     transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
                                     transforms.ToTensor(),   # Converting image to tensor
                                     transforms.Normalize(mean, std), # Normalizing with standard mean and standard deviation
                                     transforms.RandomErasing(p=0.75,scale=(0.02, 0.1),value=1.0, inplace=False)])


composed_test = transforms.Compose([transforms.Resize((CFG.size,CFG.size)),transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])



train_dataset =  DogsDataset(label_csv,root='/train', transform = composed_train)
valid_dataset =  DogsDataset(label_csv,root='/train', transform = composed_test)

train_loader = DataLoader(train_dataset,
                          batch_size=CFG.batch_size,
                          sampler=sampler.SubsetRandomSampler(range(CFG.trian_size)),
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=True,
                          )

valid_loader = DataLoader(valid_dataset,
                          batch_size=CFG.batch_size,
                          sampler=sampler.SubsetRandomSampler(range(CFG.trian_size, len(valid_dataset))),
                          num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [10]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [11]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()
    losses = AverageMeter()

    for step, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)

        y_preds = model(images)

        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    losses = AverageMeter()
    correct = 0

    for step, (images, labels) in tqdm(enumerate(valid_loader), total=len(valid_loader)):

        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
#             score = log_loss(y_preds.cpu(), labels.cpu())
        losses.update(loss.item(), batch_size)
        _, yhat = torch.max(y_preds.data, 1)

        correct += (yhat==labels).sum().item()
        
    accuracy = correct/(len(valid_dataset)-CFG.trian_size)
    
    
    return losses.avg, accuracy

best_score = -1
best_loss = np.inf
for epoch in range(CFG.epochs):

    start_time = time.time()

        # train
    avg_loss = train_fn(train_loader, model, criterion, optimizer, device)

    # eval
    avg_val_loss, score = valid_fn(valid_loader, model, criterion, device)

    scheduler.step(avg_val_loss)
    

    elapsed = time.time() - start_time

    Logger.info(
        f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
    # Logger.info(f'Epoch {epoch+1} - avgScore: {avg_score:.4f}')
    Logger.info(
        f'Epoch {epoch+1} - avgScore: {score:.4f}')
    
    update_best = score > best_score

    if update_best:
        best_loss = avg_val_loss
        best_score = score

        Logger.info(
            f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
        Logger.info(
            f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')

        torch.save({'model': model.state_dict(),},
                    CFG.model_dir + f'{CFG.model_name}_best.pth')

  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1 - avg_train_loss: 4.9363  avg_val_loss: 4.8888  time: 95s
Epoch 1 - avgScore: 0.0135
Epoch 1 - Save Best Score: 0.0135 Model
Epoch 1 - Save Best Loss: 4.8888 Model


  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

...

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 49 - avg_train_loss: 0.3606  avg_val_loss: 1.8848  time: 87s
Epoch 49 - avgScore: 0.5676


  0%|          | 0/312 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 50 - avg_train_loss: 0.3596  avg_val_loss: 1.8750  time: 87s
Epoch 50 - avgScore: 0.5586


In [12]:

sample_path = join(CFG.input_path,'sample_submission.csv')
sample = pd.read_csv(sample_path)
print(sample.values)
id2label = {id: value for id, value in sample.values}
print(id2label)

[['000621fb3cbb32d8935728e48679680e' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]
 ['00102ee9d8eb90812350685311fe5890' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]
 ['0012a730dfa437f5f3613fb75efcd4ce' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]
 ...
 ['fff74b59b758bbbf13a5793182a9bbe4' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]
 ['fff7d50d848e8014ac1e9172dc6762a3' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]
 ['fffbff22c1f51e3dc80c4bf04089545b' 0.0083333333333333
  0.0083333333333333 ... 0.0083333333333333 0.0083333333333333
  0.0083333333333333]]


ValueError: too many values to unpack (expected 2)

In [ ]:
test_dataset =  DogsDataset(sample,root='/test', transform = composed_test)
test_loader = DataLoader(test_dataset,
                         batch_size=32,
                         shuffle = False,
                         num_workers=4, pin_memory=True, drop_last=False)

for step, (images,idx) in tqdm(enumerate(test_loader), total=len(test_loader)):
    images = images.to(device)
    batch_size = images.size(0)

    with torch.no_grad():
        y_preds = model(images)
        _, yhat = torch.max(y_preds.data, 1)
    print(y_preds)